In [ ]:
# External imports
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from pathlib import Path


# Internal imports
from pareto.data_io import get_cache, get_session, get_unit_channels, get_spike_times, get_stimulus_presentations, get_units_by_area, get_trials
from pareto.preprocessing import build_onsets_table, build_psths
from pareto.stats import visual_selectivity_filter

ROOT_DIR = Path.cwd().parents[1]
DATA_DIR = ROOT_DIR / 'data'

# Obtaining cache

cache = get_cache()

# Obtaining session

session_ids = [1044385384, 1104052767, 1112302803, 1139846596, 1104058216]
sid = 1044385384
session = get_session(sid)

# Obtaining session subdata — units, stimulus presentations, trials and spike times

stimulus_presentations = get_stimulus_presentations(session)
trials = get_trials(session)
units = get_unit_channels(session)
spike_times = get_spike_times(session)

# Defining common variables

windows = {
  "pre": (-0.25, 0),
  "evoked": (0, 0.25),
  "post": (0.25, 0.5)
}

areas_of_interest = ['VISp', 'VISpm']

/opt/anaconda3/envs/allensdk/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/allensdk/lib/python3.11/site-packages/hdmf/spec/namespace.py:583: UserWarning: Ignoring the following cached namespace(s) because another version is already loaded:
core - cached version: 2.6.0-alpha, loaded version: 2.7.0
The loaded extension(s) may not be compatible with the cached extension(s) in the file. Please check the extension documentation and ignore this warning if these versions are compatible.
  self.warn_for_ignored_namespaces(ignored_namespaces)


Index(['PT_ratio', 'amplitude', 'amplitude_cutoff', 'cluster_id',
       'cumulative_drift', 'd_prime', 'firing_rate', 'isi_violations',
       'isolation_distance', 'l_ratio', 'local_index', 'max_drift',
       'nn_hit_rate', 'nn_miss_rate', 'peak_channel_id', 'presence_ratio',
       'quality', 'recovery_slope', 'repolarization_slope', 'silhouette_score',
       'snr', 'spread', 'velocity_above', 'velocity_below',
       'waveform_duration', 'anterior_posterior_ccf_coordinate',
       'dorsal_ventral_ccf_coordinate', 'filtering',
       'left_right_ccf_coordinate', 'probe_channel_number',
       'probe_horizontal_position', 'probe_id', 'probe_vertical_position',
       'structure_acronym'],
      dtype='object')


In [2]:
# Building onsets table

o_p = DATA_DIR / 'onsets' / f'{sid}.csv'

if not o_p.exists():
  onsets_table = build_onsets_table(sid, stimulus_presentations, trials)
  onsets_table.to_csv(o_p)
else:
   onsets_table = pd.read_csv(o_p)

In [3]:
# Filtering units for specified areas of interest, quality metric and visual selectivity

area_units = get_units_by_area(units, areas_of_interest)

quality_unit_filter = ((area_units['snr'] >= 1) & (area_units['isi_violations'] < 1) & (area_units['firing_rate'] > 0.1))
filtered_area_units = area_units.loc[quality_unit_filter].copy()

u_p = DATA_DIR / 'units' / f'{sid}.csv'

if not u_p.exists():
  au = filtered_area_units['structure_acronym']
  au.to_csv(u_p)


In [ ]:


s_p = DATA_DIR / 'wilcoxon_selectivity' / f'{sid}.csv'

if not s_p.exists():
    
    rows = []
    
    for img, grp in onsets_table.groupby("image_name"):
       
      onsets = grp["start_time"]
      mask, stats = visual_selectivity_filter(filtered_area_units, spike_times, onsets, windows["pre"], windows["evoked"])
      stats = stats.copy()
      stats["unit_id"] = stats.index
      stats.loc[:, "image_name"] = img
      rows.append(stats)

    vsel = pd.concat(rows, ignore_index=True)
    vsel.to_csv(s_p)

In [4]:
# Building PSTH array with units filtered for quality metrics // visual selectivity filters to be used after

p_p = DATA_DIR / 'psths' / f"{sid}.npy"

if not p_p.exists():
  psths, tps = build_psths(filtered_area_units, spike_times, onsets_table, windows, bin_size=0.01)
  np.save(p_p, psths)

tp_p = DATA_DIR / 'tps' / 'tps.npy'

if not tp_p.exists():
   np.array(tps)
   np.save(tp_p, tps)